In [10]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd

from pymatgen.transformations.standard_transformations import SupercellTransformation

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

import os
#os.getcwd()
#os.chdir('data')

In [11]:
fe_cif_filepath = "data/fe.cif"
fe_structure_2atom = from_file(structure_file=fe_cif_filepath)
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)
fe_structure_4atom = rotate_cell_45_degrees.apply_transformation(fe_structure_2atom)
print(fe_structure_4atom)

Full Formula (Fe4)
Reduced Formula: Fe
abc   :   4.053843   4.053843   2.866500
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b     c
---  ----  ---  ---  ----
  0  Fe    0    0     0
  1  Fe    0.5  0.5  -0
  2  Fe    0.5  0     0.5
  3  Fe    1    0.5   0.5


Use the 4 atom basis to generate the n = NxNxN supercell configuration. 
The neighbor information is extracted from the results of *count_neighbors* . The final output is the total number of sites,  4n  , the count of nearest neighbors for each site within the radius, r , and a list of the indices for the nearest neighbors of each site _i_.

This information can be used as the input to the Monte Carlo Simulation, together with the calculated exchange parameters. 

In [12]:
N = 10
r = 3
fe_structure_supercell = fe_structure_4atom.copy()
fe_structure_supercell.make_supercell([N,N,N])
fe_neighbor_data = count_neighbors(fe_structure_supercell, r)
a = fe_neighbor_data.data_frame \
    .merge(fe_neighbor_data.data_frame.rename(columns={"j": "i", "i": "j"}), how="outer") \
    .sort_values(["i","distance_bin", "j"]) \
    .loc[:, ["i", "j", "distance_bin"]] \
    .reset_index(drop=True)

In [13]:
neighbors_per_site = a.groupby(["i"]).count().reset_index().rename(columns={"j": "count"}).loc[:, "count"].values
sites_per_distance_group = a.groupby(["i", "distance_bin"]).count().reset_index() \
                            .rename(columns={"j": "count"}).loc[:, "count"].values
neighbor_indices = a["j"].values

In [14]:
#neighbor_count, n_indices = np.unique(sites_per_distance_group, return_index=True)
neighbor_count = np.dstack ( np.unique(sites_per_distance_group, return_index=True) )    
neighbor_count.dtype = np.dtype([('v', neighbor_count.dtype), ('i', neighbor_count.dtype)])
neighbor_count.sort(order='i', axis=1)
neighbor_count = neighbor_count.flatten()['v'].tolist()

To be used in the MC simulation:

In [15]:
print("Number of sites:", 4*N*N*N)
print("Number of 1st and 2nd neighbors for each index i:", neighbor_count)
print("Neighbor indices sorted over index i and distances: ", neighbor_indices)

Number of sites: 4000
Number of 1st and 2nd neighbors for each index i: [8, 6]
Neighbor indices sorted over index i and distances:  [2000 2009 2900 ... 2999 3990 3998]


In [16]:
filename = os.getcwd()+'/neighbor_lists.txt'

n_sites = np.array([4*N*N*N])
#np.savetxt(filename,n_sites,fmt='%d' )
#np.savetxt(filename,neighbor_count,fmt='%d' )
f=open(filename,'w')
#with open(filename,"w") as f:
f.write("\n".join(" ".join(map(str, x)) for x in (n_sites,neighbor_count)))
#f.write("\n")
#f.write(str(neighbor_indices))
#f.close()
#f=open(filename,'a')
np.savetxt(filename, neighbor_indices,fmt='%d')
f.close()


In [17]:
#os.getcwd()
#os.chdir('/home/ssilayi/Projects/spyns/Fe/exchange_model')